# Mosaic による地理空間データの分散処理

このエクササイズでは、**行ごとにポリゴン情報が含まれる** [GeoJSONSeq](https://gdal.org/drivers/vector/geojsonseq.html) ファイルを Spark でロードし、各ワーカー上で分散処理を行います。

[Mosaic](https://databrickslabs.github.io/mosaic/index.html) は、非常に大きな地理空間データセットを簡単かつ高速に処理できるようにするApache Sparkフレームワークの拡張機能です。

Mosaic は以下を提供します：
- 一般的な空間データ エンコーディング (WKT、WKB、および GeoJSON) 間の簡単な変換。
- Spark ネイティブ データ型から新しいジオメトリを簡単に生成するコンストラクター。
- 空間データセットを変換、集約、および結合するための Spark Expressions として実装された OGC SQL 標準ST_関数。
- コア Mosaic 関数内での Spark コード生成の実装による高性能。
- Ordnance Survey (ブログ投稿)と共同開発したアプローチを使用して、ポイント イン ポリゴン結合を実行するための最適化。
- Scala、SQL、および Python API の選択。


`spark.read.json` が求めるファイルは一般的な JSON ファイルではないことに注意してください。各行は別個の自己内包の有効な JSON オブジェクトでなければなりません。これは [JSON Lines](https://jsonlines.org/) や [Newline-Delimited JSON](http://ndjson.org/) とも呼ばれます。

https://databrickslabs.github.io/mosaic/index.html

In [0]:
from pyspark.sql.functions import *

In [0]:
from mosaic import enable_mosaic
enable_mosaic(spark, dbutils)

In [0]:
from mosaic import st_point

lons = [-80., -80., -70., -70., -80.]
lats = [ 35.,  45.,  45.,  35.,  35.]

bounds_df = (
  spark
  .createDataFrame({"lon": lon, "lat": lat} for lon, lat in zip(lons, lats))
  .coalesce(1)
  .withColumn("point_geom", st_point("lon", "lat"))
)
bounds_df.show()

+----+-----+--------------------+
 lat| lon| point_geom|
+----+-----+--------------------+
35.0|-80.0|{1, 0, [[[-80.0, ...|
45.0|-80.0|{1, 0, [[[-80.0, ...|
45.0|-70.0|{1, 0, [[[-70.0, ...|
35.0|-70.0|{1, 0, [[[-70.0, ...|
35.0|-80.0|{1, 0, [[[-80.0, ...|
+----+-----+--------------------+

## JSON ファイルのロード

In [0]:
from mosaic import st_geomfromgeojson

geoJsonDF = (
  spark.read.format("json")
  .load("/mnt/testblob/geojson/niigata_cropfiled_geojsonl.json")
  .withColumn("geometry", st_geomfromgeojson(to_json(col("geometry"))))
  .select("properties.*", "geometry")
  .drop("shape_area", "shape_leng")
)

geoJsonDF.show()

+---------+----------+---------+--------------------+-----------+-------------------+--------------------+------------+-------------+--------------------+--------------------+
edit_year|issue_year|land_type| last_polygon_uuid| layer|local_government_cd| path| point_lat| point_lng| polygon_uuid| geometry|
+---------+----------+---------+--------------------+-----------+-------------------+--------------------+------------+-------------+--------------------+--------------------+
 2020| 2022| 100|e17bca66-87cf-4d7...|2022_151017| 151017|/Users/han/work/S...|37.917619139|139.251546719|ccccbc5d-3b2c-4a0...|{5, 4326, [[[139....|
 2020| 2022| 200|218aac09-72f3-4d7...|2022_151017| 151017|/Users/han/work/S...|37.864208476|139.232014575|1acc7fb0-9c51-4dc...|{5, 4326, [[[139....|
 2020| 2022| 100|3cb10bab-5a07-474...|2022_151017| 151017|/Users/han/work/S...|37.862469773|139.187742061|82e377ec-f41c-4fc...|{5, 4326, [[[139....|
 2020| 2022| 200|9bb4045d-deb5-437...|2022_151017| 151017|/Users/han/work/S...|37.955433882|139.227054847|f9b74f60-c6fc-4d4...|{5, 4326, [[[139....|
 2020| 2022| 100|73c08a4e-9816-4f4...|2022_151017| 151017|/Users/han/work/S...|37.934490886|139.197689946|cc94d24c-e029-4bd...|{5, 4326, [[[139....|
 2020| 2022| 100|73435e88-26db-496...|2022_151017| 151017|/Users/han/work/S...|37.932256085|139.191676797|09258d7b-9273-424...|{5, 4326, [[[139....|
 2020| 2022| 100|18b1e658-74ba-430...|2022_151017| 151017|/Users/han/work/S...|37.935895161|139.198618329|55bea87f-9b10-4ec...|{5, 4326, [[[139....|
 2020| 2022| 100|194710d2-8f88-448...|2022_151017| 151017|/Users/han/work/S...|37.934634864|139.192650631|a18b458b-c625-4b5...|{5, 4326, [[[139....|
 2020| 2022| 100|cd8f368d-dac2-408...|2022_151017| 151017|/Users/han/work/S...|37.935734401|139.204146018|6d98a1c1-fcee-418...|{5, 4326, [[[139....|
 2020| 2022| 100|832db4d1-045d-457...|2022_151017| 151017|/Users/han/work/S...|37.937496937|139.205585293|578cfc7a-a096-4a7...|{5, 4326, [[[139....|
 2020| 2022| 100|d536f366-5337-478...|2022_151017| 151017|/Users/han/work/S...|37.937137055|139.204797405|73550212-34ab-474...|{5, 4326, [[[139....|
 2020| 2022| 100|dfd449fc-7a2f-49a...|2022_151017| 151017|/Users/han/work/S...|37.934885981|139.203453871|effa777a-052d-42d...|{5, 4326, [[[139....|
 2020| 2022| 100|3a6a594f-4052-4f5...|2022_151017| 151017|/Users/han/work/S...|37.937046635|139.208434762|8d9a5b59-0a51-434...|{5, 4326, [[[139....|
 2020| 2022| 100|ed5cdb19-1673-445...|2022_151017| 151017|/Users/han/work/S...|37.935272307|139.202463398|c2bfe99d-a8bc-4db...|{5, 4326, [[[139....|
 2020| 2022| 100|6237bf8d-3ff9-4c0...|2022_151017| 151017|/Users/han/work/S...|37.937519555|139.203793519|3724c497-89a7-485...|{5, 4326, [[[139....|
 2020| 2022| 100|69ea3957-8981-455...|2022_151017| 151017|/Users/han/work/S...|37.938509591|139.203586419|81d92714-49a2-4cf...|{5, 4326, [[[139....|
 2020| 2022| 100|22006551-386b-4bf...|2022_151017| 151017|/Users/han/work/S...|37.936482997|139.203991097|0eacf5ef-8183-409...|{5, 4326, [[[139....|
 2020| 2022| 100|14a4eeca-3965-4e5...|2022_151017| 151017|/Users/han/work/S...| 37.93596584| 139.20589768|cff9a34b-3154-493...|{5, 4326, [[[139....|
 2020| 2022| 100|fed6ac06-96a0-415...|2022_151017| 151017|/Users/han/work/S...|37.940211893|139.207784394|1b62e76d-2237-4f0...|{5, 4326, [[[139....|
 2020| 2022| 100|bf2cd9e6-0e83-43d...|2022_151017| 151017|/Users/han/work/S...|37.936934581| 139.20754169|ee0a2f37-efc0-496...|{5, 4326, [[[139....|
+---------+----------+---------+--------------------+-----------+-------------------+--------------------+------------+-------------+--------------------+--------------------+
only showing top 20 rows

## データの取得
PySparkでは、アクションを使用してDataFrame `show()` の先頭/最初の N (5,10,100 ..) 行を取得し、それらをコンソールまたはログに表示できます。行のリストとしての先頭と最後の n 行 (Scala の場合は Array[Row])。Spark アクションは Spark Driver に結果を取得するため、大きなデータセットを抽出するときは十分に注意する必要があります。

https://sparkbyexamples.com/spark/show-top-n-rows-in-spark-pyspark/

In [0]:
geoJsonDF.take(5)

Out[28]: [Row(edit_year=2020, issue_year=2022, land_type=100, last_polygon_uuid='e17bca66-87cf-4d75-a357-dcec49a17d48', layer='2022_151017', local_government_cd='151017', path='/Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017', point_lat=37.917619139, point_lng=139.251546719, polygon_uuid='ccccbc5d-3b2c-4a06-9d8b-aef5b930daf9', geometry=Row(type_id=5, srid=4326, boundary=[[[139.2514364, 37.9174405], [139.2518658, 37.9176022], [139.2515764, 37.9178179], [139.2515616, 37.9178094], [139.2515459, 37.9178175], [139.2512581, 37.9175647], [139.2514364, 37.9174405]]], holes=[[]])),
 Row(edit_year=2020, issue_year=2022, land_type=200, last_polygon_uuid='218aac09-72f3-4d7b-9459-e6c275330465', layer='2022_151017', local_government_cd='151017', path='/Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017', point_lat=37.864208476, point_lng=139.232014575, polygon_uuid='1acc7fb0-9c51-4dc6-ae4c-9e63cbaa3632', geometry=Row(type_id=5, srid=4326, boundary=[[[139.2318403, 37.8640836], [139.232175, 37.8642189], [139.2323779, 37.8643338], [139.2317783, 37.8642065], [139.2318403, 37.8640836]]], holes=[[]])),
 Row(edit_year=2020, issue_year=2022, land_type=100, last_polygon_uuid='3cb10bab-5a07-4745-b0ee-a0fa9c46a2b0', layer='2022_151017', local_government_cd='151017', path='/Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017', point_lat=37.862469773, point_lng=139.187742061, polygon_uuid='82e377ec-f41c-4fc6-b8e2-f561fc71d64a', geometry=Row(type_id=5, srid=4326, boundary=[[[139.1877035, 37.8626389], [139.1876241, 37.8623294], [139.1878656, 37.8623795], [139.1877831, 37.8626259], [139.1877035, 37.8626389]]], holes=[[]])),
 Row(edit_year=2020, issue_year=2022, land_type=200, last_polygon_uuid='9bb4045d-deb5-437f-b01f-a607cdd19d7e', layer='2022_151017', local_government_cd='151017', path='/Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017', point_lat=37.955433882, point_lng=139.227054847, polygon_uuid='f9b74f60-c6fc-4d43-80c5-5e5e4110315c', geometry=Row(type_id=5, srid=4326, boundary=[[[139.2270315, 37.9552805], [139.2272117, 37.9553614], [139.2270723, 37.9555833], [139.2269022, 37.9555181], [139.2270315, 37.9552805]]], holes=[[]])),
 Row(edit_year=2020, issue_year=2022, land_type=100, last_polygon_uuid='73c08a4e-9816-4f44-b28c-8df7d53da99d', layer='2022_151017', local_government_cd='151017', path='/Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017', point_lat=37.934490886, point_lng=139.197689946, polygon_uuid='cc94d24c-e029-4bdf-9df3-9e51667b4648', geometry=Row(type_id=5, srid=4326, boundary=[[[139.1974306, 37.9342118], [139.197443, 37.9342136], [139.1981092, 37.9345739], [139.1979453, 37.9347665], [139.1979058, 37.9347524], [139.197312, 37.9344327], [139.1972971, 37.9344069], [139.1973069, 37.934364], [139.1974306, 37.9342118]]], holes=[[]]))]

In [0]:
geoJsonDF.count()

Out[23]: 1312844

In [0]:
from mosaic import st_area, st_length
(
  geoJsonDF
  .withColumn("calculatedArea", abs(st_area("geometry")))
  .withColumn("calculatedLength", st_length("geometry"))
  .select("geometry", "calculatedArea", "calculatedLength")
).show()

+--------------------+--------------------+--------------------+
 geometry| calculatedArea| calculatedLength|
+--------------------+--------------------+--------------------+
{5, 4326, [[[139....|1.157189649996655...|0.001454867922257542|
{5, 4326, [[[139....|4.629405000172204...|0.001344805070674...|
{5, 4326, [[[139....|4.465362500081416...|9.066635993295671E-4|
{5, 4326, [[[139....|5.005497999734362...|9.122515454265145E-4|
{5, 4326, [[[139....|1.936108949997856...|0.002009080412180693|
{5, 4326, [[[139....|1.859641799956153E-7|0.001928051588067...|
{5, 4326, [[[139....|1.855006250020834...|0.001955288220459...|
{5, 4326, [[[139....|1.854488449959949...|0.002021404363896...|
{5, 4326, [[[139....|1.985785299981092...|0.002092566401772685|
{5, 4326, [[[139....|1.985133699982610...|0.002088711424482...|
{5, 4326, [[[139....|1.985946699980149...|0.002110829632118498|
{5, 4326, [[[139....| 1.98219400002284E-7|0.002093332821736...|
{5, 4326, [[[139....|1.990338999984252E-7|0.002120929130548...|
{5, 4326, [[[139....|1.988024099987779...|0.002106174878095...|
{5, 4326, [[[139....|1.988322099926919...|0.002100534641283...|
{5, 4326, [[[139....|1.994700799927773...|0.002084322090008...|
{5, 4326, [[[139....|1.993273799993734...|0.002116260029440442|
{5, 4326, [[[139....|1.990930350004482E-7|0.002130748314290231|
{5, 4326, [[[139....|1.965042449998458...|0.002090689740813681|
{5, 4326, [[[139....|1.965834350007009...|0.002091369632205552|
+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
# RDD のパーティション数を返します
geoJsonDF.rdd.getNumPartitions()

Out[21]: 8